In [ ]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
data = pd.read_csv('data/reloading_data - 9mm parabellum.csv')
data.head()

In [ ]:
data.shape

In [ ]:
# add a "bullet type" column

orig_columns =                           ['powder', 'start', 'velocity', 'never exceed', 'velocity max', 'press', 'units', 'min oal']
new_columns  = ['bul_weight', 'bul_type', 'powder', 's_load', 's_velocity', 'ne_load', 'ne_velocity', 'press', 'units', 'oal']

lee = pd.DataFrame(columns = new_columns)

bul_weight = None
bul_type = None
for pos in range(0, data.shape[0]):
    line = tuple(data.loc[pos, orig_columns].array)
    # print(123, line) # DEBUG
    if ' grain ' in line[0]: # this is a bullettype "115 grain jacketed"
        bul_weight, bul_type = line[0].split(' grain ')
    else:
        line = (bul_weight, bul_type, *line)
        lee.loc[len(lee.index)] = line


In [ ]:
lee

In [ ]:
lee.shape

In [ ]:
powder_labels = lee.powder.unique()
print(powder_labels)
bul_types = lee.bul_type.unique()
print(bul_types)

In [ ]:
def powder_id(x):
    return np.where(lee.powder.unique() == x)[0][0]

def bul_type_id(x):
    return np.where(lee.bul_type.unique() == x)[0][0]

In [ ]:
lee['powder_id'] = [ powder_id(x) for x in lee.powder ]
lee['bul_type_id'] = [ bul_type_id(x) for x in lee.bul_type ]

In [ ]:
list(lee.bul_type_id)